In [1]:
import pandas as pd
import numpy as np

dpath = "./"
Train = pd.read_csv(dpath + "FE_train_tfidf.csv")
Train.head()

,0,1,2,3,4,5,6,7,8,9,...,3595,3596,3597,3598,3599,3600,3601,3602,3603,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [2]:
y_train = Train["Label"]
X_train = Train.drop(["Label"], axis=1)

# 数据归一化:每个样本的模长为1
from sklearn import preprocessing
preprocessing.normalize(X_train, norm="l2", copy=False)

# 稀疏矩阵输入
from scipy.sparse import csr_matrix
X_train = csr_matrix(X_train)

In [3]:
# 训练数据
from sklearn.cluster import MiniBatchKMeans as KMeans
from sklearn import metrics
# 一个参数点聚类函数
def K_cluster_analysis(K, X):
    print("K-means begin with clusters: {}".format(K))
    
    # K-means,在训练集上训练
    mb_kmeans = KMeans(n_clusters = K)
    y_pred = mb_kmeans.fit_predict(X)
    
    # CN作为评价指标
    CN_score = metrics.calinski_harabaz_score(X.todense(), y_pred)
    
    print("CN_score: {}".format(CN_score))
    return CN_score

In [4]:
# 聚类搜索范围
Ks = [5, 10, 15, 20, 30, 40, 50]
CN_scores = []
for K in Ks:
    ch = K_cluster_analysis(K, X_train)
    CN_scores.append(ch)

K-means begin with clusters: 5
CN_score: 45.4299355637594
K-means begin with clusters: 10
CN_score: 30.728942227539434
K-means begin with clusters: 15
CN_score: 23.274547683555653
K-means begin with clusters: 20
CN_score: 19.57552689518874
K-means begin with clusters: 30
CN_score: 16.003550071816527
K-means begin with clusters: 40
CN_score: 12.370534662277686
K-means begin with clusters: 50
CN_score: 10.363448824850954


In [5]:
# 绘制不同K对应的聚类性能，找到最佳参数
import matplotlib.pyplot as plt
import seaborn as sns

plt.plot(Ks, np.array(CN_scores), "b-", label="CN_scores")

index = np.unravel_index(np.argmax(CN_scores, axis=None), len(CN_scores))
Best_K = Ks[index[0]]

plt.show()
print(Best_K)

<Figure size 640x480 with 1 Axes>

5


In [6]:
# 用最佳KMeans模型
mb_kmeans = KMeans(n_clusters = 10)
y_pred = mb_kmeans.fit_predict(X_train)

y_pred

array([9, 6, 2, ..., 8, 8, 8])

In [8]:
# 保存聚类结果
feat_names_Kmeans = "KMeans_" + str(Best_K)